In [3]:
import cobra
from cobra.io import load_model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.io import write_sbml_model, validate_sbml_model

#### LEER MEDICAGO MODEL

In [4]:
%cd C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Medicago truncatula
medModel=read_sbml_model("MedicagoTruncatula_Cobra_model.xml")


C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Medicago truncatula


In [8]:

medModel.objective="BiomassShoot"
solution=medModel.optimize()
solution

,fluxes,reduced_costs
TEC_AMMONIUM,0.000000e+00,0.0
TEC_CARBON-DIOXIDE,0.000000e+00,0.0
TEC_CO+2,0.000000e+00,0.0
TEC_CPD-3,0.000000e+00,0.0
TEC_FE+2,0.000000e+00,0.0
...,...,...
URATE-OXIDASE-RXN_X,0.000000e+00,0.0
DEHOG,-1.000000e+03,0.0
BiomassShoot,0.000000e+00,2.0
BiomassRoot,5.032460e-16,0.0


In [9]:
for i in medModel.metabolites.get_by_id("ASP-tRNAs[C]").reactions:
    print(i.id+":   " +i.reaction)

BiomassWithOutStarch:   1226.94420253165 ASN[C] + 3.11908860759494 ATP[C] + 64.4638987341772 BETA-D-FRUCTOSE[C] + 1276.87098734177 CELLULOSE[G] + 3.69883544303797 CHLOROPHYLL-A[H] + 2.04536708860759 CHLOROPHYLL-B[H] + 10.7046075949367 CIT[C] + 303.191696202532 CONIFERYL-ALCOHOL[C] + 28.6394936708861 COUMARYL-ALCOHOL[C] + 23.4783797468354 CPD-8046[C] + 0.64539746835443 CPD1F-114[H] + 3.11908860759494 CTP[C] + 123.521721518987 Charged-ALA-tRNAs[C] + 31.5128101265823 Charged-ARG-tRNAs[C] + 69.4923544303797 Charged-ASN-tRNAs[C] + 69.5044050632911 Charged-ASP-tRNAs[C] + 28.0813164556962 Charged-CYS-tRNAs[C] + 60.9379240506329 Charged-GLN-tRNAs[C] + 60.9479493670886 Charged-GLT-tRNAs[C] + 154.282835443038 Charged-GLY-tRNAs[C] + 30.4581265822785 Charged-HIS-tRNAs[C] + 4.93812658227848 Charged-ILE-tRNAs[C] + 49.1159493670886 Charged-LEU-tRNAs[C] + 68.0082025316456 Charged-LYS-tRNAs[C] + 22.8016202531646 Charged-MET-tRNAs[C] + 53.5664810126582 Charged-PHE-tRNAs[C] + 37.0576202531646 Charged-PRO

#### LEER PVUL MODEL

In [3]:
%cd C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris
pvulModel=read_sbml_model('submodel.xml')

C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


#### METABOLITOS ASOCIADOS A LAS RXNS DE BIOMASA EN MEDICAGO

In [6]:
print(len(medModel.reactions.get_by_id("BiomassShoot").metabolites))
print(len(medModel.reactions.get_by_id("BiomassRoot").metabolites))

96
91


#### CREAR DATAFRAME DE LOS METS CON SUS CONCENTRACIONES PARA MABAS RXNS DE BIOMASA

In [ ]:
import pandas as pd
df=pd.DataFrame()
df2=pd.DataFrame()
for i in medModel.reactions.get_by_id("BiomassShoot").metabolites:
    df=df.append({'Biomass Shoot metabolite':i.id.replace("[C]","[c]"),'coefficient Shoot':medModel.reactions.get_by_id("BiomassShoot").metabolites[i]},ignore_index=True)
for j in medModel.reactions.get_by_id("BiomassRoot").metabolites:
    coeff = medModel.reactions.get_by_id("BiomassRoot").metabolites[j]
    if coeff is None:
        print(f"No coefficient found for {j.id}")
    df2=df2.append({'Biomass Root metabolite':j.id.replace("[C]","[c]"),'coefficient Root':medModel.reactions.get_by_id("BiomassRoot").metabolites[j]},ignore_index=True)
Dataframe=pd.concat([df,df2],axis=1)

In [8]:
Dataframe.to_csv('BiomassMetabolites.csv',index=False)

#### SACAR METS UNICOSS PARA CADA BIOMASS RXNS

In [9]:
uniqueShoot=set(Dataframe['Biomass Shoot metabolite'].unique()) - set(Dataframe['Biomass Root metabolite'].unique())
uniqueRoot=set(Dataframe['Biomass Root metabolite'].unique()) - set(Dataframe['Biomass Shoot metabolite'].unique())
print(uniqueShoot)
print(uniqueRoot)

{'CHLOROPHYLL-A[H]', 'Starch[H]', 'SER[c]', 'CHLOROPHYLL-B[H]', 'CPD1F-114[H]'}
{nan}


In [10]:
medMetsBiomas=[]
for i in medModel.reactions.get_by_id("BiomassShoot").metabolites:
      r=medModel.metabolites.get_by_id(i.id)
      medMetsBiomas.append(r.id.replace("[C]","[c]"))
len(medMetsBiomas)

96

#### METS  DE RXNSS DE BIOMASA DE SHOOT DE MEDICAGO QUE SE ENCUENTRAN EN RXNS DE PVULMODEL

In [ ]:
containsMeds=[pvulMet for medMet in medMetsBiomas for pvulMet in pvulModel.metabolites if medMet in pvulMet.id]
print(len(containsMeds))
containsMeds
dfmetsShootBiomassMedicagoInPvulModel=pd.DataFrame()
for i in containsMeds:
    dfmetsShootBiomassMedicagoInPvulModel=dfmetsShootBiomassMedicagoInPvulModel.append({'Metabolite':i.id},ignore_index=True)
dfmetsShootBiomassMedicagoInPvulModel

In [12]:
len(dfmetsShootBiomassMedicagoInPvulModel)

45

In [13]:
differenceShoot=set(Dataframe['Biomass Shoot metabolite']) - set(dfmetsShootBiomassMedicagoInPvulModel['Metabolite'])
len(differenceShoot)

60

#### CREAR RXN DE BIOMASA

In [164]:
# rxnBiomass=Reaction('Biomass')
# rxnBiomass.name='Biomass'
# rxnBiomass.add_metabolites({
#     pvulModel.metabolites.get_by_id('PHE[c]'): -7.6969278,
#     pvulModel.metabolites.get_by_id('ATP[c]'): -2.69447148,
#     #pvulModel.metabolites.get_by_id('DATP[c]'): -6.71732676,
#     pvulModel.metabolites.get_by_id('VAL[c]'): -9.24497388,
#     pvulModel.metabolites.get_by_id('WATER[c]'): -1420.45982424,
#     pvulModel.metabolites.get_by_id('BETA-D-FRUCTOSE[c]'): -55.68810588,
#     #pvulModel.metabolites.get_by_id('DCTP[c]'): -4.11707124,
#     pvulModel.metabolites.get_by_id('SUC[c]'): -10.4070582,
#     pvulModel.metabolites.get_by_id('GLC[c]'): -44.96699448,
#     pvulModel.metabolites.get_by_id('GLT[c]'): -85.75296984,
#     pvulModel.metabolites.get_by_id('STEARIC_ACID[c]'): -10.90403208,
#     pvulModel.metabolites.get_by_id('L-ASPARTATE[c]'): -3.27096468,
#     pvulModel.metabolites.get_by_id('ASN[c]'): -1059.91415352,
#     pvulModel.metabolites.get_by_id('MYO-INOSITOL[c]'): -21.03360372,
#     pvulModel.metabolites.get_by_id('CIT[c]'): -9.24733584,
#     pvulModel.metabolites.get_by_id('GTP[c]'): -2.69447148,
#     pvulModel.metabolites.get_by_id('MAL[c]'): -16.42069584,
#     pvulModel.metabolites.get_by_id('CTP[c]'): -2.69447148,
#     pvulModel.metabolites.get_by_id('DCTP[c]'): -4.11707124,
#     pvulModel.metabolites.get_by_id('SER[c]'): -9.28433988,
#     pvulModel.metabolites.get_by_id('GLY[c]'): -12.8691828,
#     pvulModel.metabolites.get_by_id('LINOLEIC_ACID[c]'): -66.57210504,
#     pvulModel.metabolites.get_by_id('OLEATE-CPD[c]'): -19.0645164,
#     pvulModel.metabolites.get_by_id('PALMITATE[c]'): -22.7417382,
#     pvulModel.metabolites.get_by_id('PRO[c]'): -2.88841464,
#     #pvulModel.metabolites.get_by_id('UTP[c]'): -2.69447148,
#     pvulModel.metabolites.get_by_id('SUCROSE[c]'): -80.23280688,
#     #pvulModel.metabolites.get_by_id('TTP[c]'): -6.71732676,
#     pvulModel.metabolites.get_by_id('GDP-L-GALACTOSE[c]'): -56.81677284,
#     pvulModel.metabolites.get_by_id('THR[c]'): -4.58360208,
#     pvulModel.metabolites.get_by_id('ILE[c]'): -2.67898752,
#     #pvulModel.metabolites.get_by_id('DATP[c]'): -6.71732676,
#     pvulModel.metabolites.get_by_id('_5-OXOPROLINE[c]'): -5.7356262,
#     pvulModel.metabolites.get_by_id('GLN[c]'): -2.69447148,
#     pvulModel.metabolites.get_by_id('PROTON[c]'): 774.167119559999,
#     pvulModel.metabolites.get_by_id('PPI[c]'): 32.44650696,
#     pvulModel.metabolites.get_by_id('GDP[c]'): 56.81677284,
#     #pvulModel.metabolites.get_by_id('UDP[c]'): 345.79470564,
# })
# rxnBiomass.reaction




'1059.91415352 ASN[c] + 2.69447148 ATP[c] + 55.68810588 BETA-D-FRUCTOSE[c] + 9.24733584 CIT[c] + 2.69447148 CTP[c] + 4.11707124 DCTP[c] + 56.81677284 GDP-L-GALACTOSE[c] + 44.96699448 GLC[c] + 2.69447148 GLN[c] + 85.75296984 GLT[c] + 12.8691828 GLY[c] + 2.69447148 GTP[c] + 2.67898752 ILE[c] + 3.27096468 L-ASPARTATE[c] + 66.57210504 LINOLEIC_ACID[c] + 16.42069584 MAL[c] + 21.03360372 MYO-INOSITOL[c] + 19.0645164 OLEATE-CPD[c] + 22.7417382 PALMITATE[c] + 7.6969278 PHE[c] + 2.88841464 PRO[c] + 9.28433988 SER[c] + 10.90403208 STEARIC_ACID[c] + 80.23280688 SUCROSE[c] + 10.4070582 SUC[c] + 4.58360208 THR[c] + 6.71732676 TTP[c] + 2.69447148 UTP[c] + 9.24497388 VAL[c] + 1420.45982424 WATER[c] + 5.7356262 _5-OXOPROLINE[c] --> 56.81677284 GDP[c] + 32.44650696 PPI[c] + 774.167119559999 PROTON[c] + 345.79470564 UDP[c]'

In [6]:
#Revissar los flujos de entrada y salida para optimizar biomasa
pvulModel.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ASN[e],EX_ASN[e],508.3,4,6.32%
BETA-D-FRUCTOSE[e],EX_BETA-D-FRUCTOSE[e],5.142,6,0.10%
CIT[e],EX_CIT[e],366,6,6.82%
GLN[e],EX_GLN[e],1000,5,15.54%
GLY[e],EX_GLY[e],428.8,2,2.67%
ILE[e],EX_ILE[e],1.285,6,0.02%
L-ASPARTATE[e],EX_L-ASPARTATE[e],1.569,4,0.02%
LINOLEIC_ACID[e],EX_LINOLEIC_ACID[e],31.93,18,1.79%
LYS[e],EX_LYS[e],308.9,6,5.76%
MAL[e],EX_MAL[e],1000,4,12.43%


In [ ]:
#ELiminar rxn 
removerxn=pvulModel.reactions.get_by_id("imaginary2")
pvulModel.remove_reactions([removerxn])

In [4]:
#Creacion de rxn de biomasa
imaginary=Reaction('Biomass')
imaginary.name='Biomass'
imaginary.add_metabolites({
    pvulModel.metabolites.get_by_id('PHE[c]'): -7.6969278,
    pvulModel.metabolites.get_by_id('ATP[c]'): -2.69447148,
    pvulModel.metabolites.get_by_id('VAL[c]'): -9.24497388,
    pvulModel.metabolites.get_by_id('WATER[c]'): -1420.45982424,
    pvulModel.metabolites.get_by_id('BETA-D-FRUCTOSE[c]'): -55.68810588,
    pvulModel.metabolites.get_by_id('SUC[c]'): -10.4070582,
    pvulModel.metabolites.get_by_id('GLC[c]'): -44.96699448,
    pvulModel.metabolites.get_by_id('GLT[c]'): -85.75296984,
    pvulModel.metabolites.get_by_id('STEARIC_ACID[c]'): -10.90403208,
    pvulModel.metabolites.get_by_id('L-ASPARTATE[c]'): -3.27096468,
    pvulModel.metabolites.get_by_id('ASN[c]'): -1059.91415352,
    pvulModel.metabolites.get_by_id('MYO-INOSITOL[c]'): -21.03360372,
    pvulModel.metabolites.get_by_id('CPD-406[c]'): -1400,
    pvulModel.metabolites.get_by_id('CIT[c]'): -9.24733584,
    pvulModel.metabolites.get_by_id('GTP[c]'): -2.69447148,
    pvulModel.metabolites.get_by_id('MAL[c]'): -16.42069584,
    pvulModel.metabolites.get_by_id('SER[c]'): -9.28433988,
    pvulModel.metabolites.get_by_id('GLY[c]'): -12.8691828,
    pvulModel.metabolites.get_by_id('LINOLEIC_ACID[c]'): -66.57210504,
    pvulModel.metabolites.get_by_id('OLEATE-CPD[c]'): -19.0645164,
    pvulModel.metabolites.get_by_id('PALMITATE[c]'): -22.7417382,
    pvulModel.metabolites.get_by_id('PRO[c]'): -2.88841464,
    pvulModel.metabolites.get_by_id('SUCROSE[c]'): -80.23280688,
    pvulModel.metabolites.get_by_id('GDP-L-GALACTOSE[c]'): -56.81677284,
    pvulModel.metabolites.get_by_id('THR[c]'): -4.58360208,
    pvulModel.metabolites.get_by_id('ILE[c]'): -2.67898752,
    pvulModel.metabolites.get_by_id('_5-OXOPROLINE[c]'): -5.7356262,
    pvulModel.metabolites.get_by_id('GLN[c]'): -2.69447148,
    pvulModel.metabolites.get_by_id('PROTON[c]'): 774.167119559999,
    pvulModel.metabolites.get_by_id('PPI[c]'): 32.44650696,
    pvulModel.metabolites.get_by_id('GDP[c]'): 56.81677284,
})
pvulModel.add_reactions([imaginary])

In [5]:
# settear biomasa como funcion obj
pvulModel.objective="Biomass"

In [ ]:
#chequear rxn bloqueadas
cobra.flux_analysis.find_blocked_reactions(pvulModel)

['Biomass']

In [7]:
write_sbml_model(pvulModel, 'pvulModelBiomass.xml')